Como visto no artigo "When is multitask learning effective? Semantic sequence prediction under varying data conditions", nota-se que boas tarefas auxiliares tendem a ter baixo kurtosis e alta entropia.

Vamos ver como a tarefa de emoções se sai.
E também como a tarefa primária se compara.

In [ ]:
import pandas as pd
df = pd.read_csv("data/goemotions/train.tsv",sep='\t',index_col=0)

emotion_dict = {
    "admiração": 0,
    "diversão": 1,
    "raiva": 2,
    "aborrecimento": 3,
    "aprovação": 4,
    "zelo": 5,
    "confusão": 6,
    "curiosidade": 7,
    "desejo": 8,
    "decepção": 9,
    "desaprovação": 10,
    "nojo": 11,
    "constrangimento": 12,
    "entusiasmo": 13,
    "medo": 14,
    "gratidão": 15,
    "luto": 16,
    "alegria": 17,
    "amor": 18,
    "nervosismo": 19,
    "otimismo": 20,
    "orgulho": 21,
    "percepção": 22,
    "alívio": 23,
    "remorso": 24,
    "tristeza": 25,
    "surpresa": 26,
    "neutro": 27
}

In [ ]:
def change_label_encoding(row):
    #mudar encoding para ser multirrótulo (preferível para o cálculo da entropia)
    labels = row['labels'].replace(" ","").split(",")
    new_row = {}
    for emotion in emotion_dict:
        if str(emotion_dict[emotion]) in labels:
            new_row[emotion] = True
        else:
            new_row[emotion] = False
    return new_row

In [ ]:
labels = pd.DataFrame(list(df.apply(change_label_encoding,axis=1)))

# Definindo funções

In [ ]:
from scipy.stats import entropy
from scipy.stats import kurtosis

def compute_label_entropy(labels):
    #se fosse pra fins de interpretação, labels.sum()/labels.shape[0] faz mais sentido
    #mas dá no mesmo no fim!
    counts = list(labels.sum())
    return entropy(counts)
    
    

# Avaliando no conjunto do GoEmotions

In [ ]:
#comparada com o visto no artigo, isso é uma boa entropia
compute_label_entropy(labels)

In [ ]:
#dropando o "neutro" (pode ser considerado como uma classe O do artigo original)
#como esperado, a entropia está um pouco maior.
compute_label_entropy(labels.drop(columns="neutro"))

In [ ]:
#kurtosis de cada label. Valores beeeem altos para alguns dos rótulos
kurtosis(labels)

In [ ]:
#tirando a média:
kurtosis(labels).mean()

## Como diminuir kurtosis?

É bem possível que essa "skew" na distribuição prejudique a capacidade da tarefa ajudar. Qual o efeito de diferentes valores de cutoff de kurtosis na entropia?

In [ ]:
def filter_by_kurtosis(labels,cutoff):
    columns=[col for col in labels.columns if kurtosis(labels[col]) > cutoff]
    return labels.drop(columns=columns)

In [ ]:
#podemos observar que a entropia de fato cai, mas não cai vertiginosamente nem nada. É bem possível que essa seja uma boa ideia!

In [ ]:
compute_label_entropy(filter_by_kurtosis(labels,300))

In [ ]:
compute_label_entropy(filter_by_kurtosis(labels,200))

In [ ]:
compute_label_entropy(filter_by_kurtosis(labels,50))

In [ ]:
# de forma programática: detecção de outliers
#TODO: tentar com zscore também
from scipy.stats import zscore
from scipy.stats import iqr



def filter_by_kurtosis_iqr(labels):
    #range interquartil
    cutoff = iqr(kurtosis(labels))
    columns=[col for col in labels.columns if kurtosis(labels[col]) > cutoff]
    return labels.drop(columns=columns)

In [ ]:
compute_label_entropy(filter_by_kurtosis_iqr(labels))

# Conjunto da tarefa primária

In [ ]:
df = pd.read_csv("data/segredos_sentenças_multitask_clean.csv",index_col=0)
#limpando sintomas que não usamos mais
df = df.drop(["Postagem com possível perfil depressivo","Alteração na eficiência",
         "Alteração da funcionalidade","*",'Agitação/inquietação','Sintoma obsessivo e compulsivo','Déficit de atenção/Memória',
              'Perda/Diminuição do prazer/ Perda/Diminuição da libido'],axis=1)
labels = df.iloc[:,1:]
labels

In [ ]:
#nada mal,mesmo sem dropar instâncias neutras
compute_label_entropy(labels)

In [ ]:
compute_label_entropy(filter_by_kurtosis_iqr(labels))

In [ ]:
#kurtosis beem ruim para alguns rótulos, mesmo após filtragem
print(kurtosis(labels))
#esses são os rótulos: justamente aqueles que são mais díficeis de prever
#mas temos que lembrar que isso não é bem regra: Alteração de sono tem um kurtosis de 183 e é bem classificado
labels.columns[kurtosis(labels) > 200]

## removendo instâncias sem rótulo

In [ ]:

labels = labels.reset_index(drop=True)
labels = labels[labels.sum(axis=1) != 0]

In [ ]:
#Não afeta a entropia, a proporção continua a mesma.
compute_label_entropy(labels)

In [ ]:
#mas afeta o kurtosis
print(kurtosis(labels))

In [ ]:
#mas ainda não muda isso aqui:
compute_label_entropy(filter_by_kurtosis_iqr(labels))